In [ ]:
import random
import math
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QPushButton, QTextEdit
from PyQt5.QtCore import QTimer
import matplotlib
matplotlib.use('Qt5Agg')
from PyQt5 import QtWidgets
import matplotlib.pyplot as plt
import numpy as np

class TSPSolver:
    def __init__(self, num_cities):
        self.num_cities = num_cities
        self.cities = [(0.2, 0.3, "New York"), (0.4, 0.6, "Los Angeles"), (0.6, 0.2, "Chicago"), (0.8, 0.8, "Houston"), (0.1, 0.5, "Phoenix"),
                       (0.7, 0.4, "Philadelphia"), (0.3, 0.7, "Seattle"), (0.5, 0.1, "Miami"), (0.2, 0.9, "Dallas"), (0.6, 0.6, "Portland")]
        self.distance_matrix = self.calculate_distance_matrix()
        self.tour = list(range(num_cities))
        self.tour_length = self.calculate_tour_length()
        self.temperature = 100
        self.optimization_history = []  # Store optimization history
        self.best_tour = self.tour.copy()
        self.best_tour_length = self.tour_length
   
    def initialize_tour(self):
        # Initialize tour with New York at start and end
        self.tour = [0] + random.sample(range(1, self.num_cities), self.num_cities - 1) + [0]
        self.tour_length = self.calculate_tour_length()
        self.best_tour = self.tour.copy()
        self.best_tour_length = self.tour_length  

    def calculate_distance_matrix(self):
        distance_matrix = [[0 for _ in range(self.num_cities)] for _ in range(self.num_cities)]
        for i in range(self.num_cities):
            for j in range(i + 1, self.num_cities):
                distance_matrix[i][j] = distance_matrix[j][i] = math.sqrt(
                    (self.cities[i][0] - self.cities[j][0]) ** 2 + (self.cities[i][1] - self.cities[j][1]) ** 2)
        return distance_matrix

    def calculate_tour_length(self):
        length = 0
        for i in range(len(self.tour) - 1):
            length += self.distance_matrix[self.tour[i]][self.tour[i + 1]]
        length += self.distance_matrix[self.tour[-1]][self.tour[0]]
        return length

    def hill_climbing(self):
        new_tour = self.generate_neighbor_tour()
        new_length = self.calculate_new_tour_length(new_tour)
        if new_length < self.tour_length or random.random() < math.exp(-(new_length - self.tour_length) / self.temperature):
            self.tour = new_tour
            self.tour_length = new_length
            self.optimization_history.append((self.tour.copy(), self.tour_length))
            if self.tour_length < self.best_tour_length:
                self.best_tour = self.tour.copy()
                self.best_tour_length = self.tour_length
        self.temperature *= 0.99

    def generate_neighbor_tour(self):
        i, j = random.sample(range(1, len(self.tour) - 1), 2)
        new_tour = self.tour.copy()
        new_tour[i], new_tour[j] = new_tour[j], new_tour[i]
        return new_tour

    def calculate_new_tour_length(self, new_tour):
        length = 0
        for i in range(len(new_tour) - 1):
            length += self.distance_matrix[new_tour[i]][new_tour[i + 1]]
        length += self.distance_matrix[new_tour[-1]][new_tour[0]]
        return length

    def get_distance_matrix_string(self):
        matrix_string = "Distance Matrix:\n"
        matrix_string += "\t" + "\t".join(city[2][:3] for city in self.cities) + "\n"
        for i, row in enumerate(self.distance_matrix):
            matrix_string += self.cities[i][2][:3] + "\t" + "\t".join(f"{dist:.2f}" for dist in row) + "\n"
        return matrix_string

class TSPVisualizer(QWidget):
    def __init__(self, tsp_solver):
        super().__init__()
        self.tsp_solver = tsp_solver
        self.max_iterations = 1000
        self.current_iteration = 0
        self.initUI()

    def initUI(self):
        self.setGeometry(300, 300, 1200, 600)
        self.setWindowTitle('Traveling Salesman Problem')
        layout = QVBoxLayout()
        self.setLayout(layout)
        self.figure, self.ax = plt.subplots(1, 2, figsize=(12, 4))
        self.canvas = matplotlib.backends.backend_qt5agg.FigureCanvasQTAgg(self.figure)
        layout.addWidget(self.canvas)
        self.label = QLabel()
        layout.addWidget(self.label)
        self.start_button = QPushButton("Start")
        self.start_button.clicked.connect(self.start_optimization)
        layout.addWidget(self.start_button)
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_visualization)
        self.pause_button = QPushButton("Pause")
        self.pause_button.clicked.connect(self.pause_optimization)
        layout.addWidget(self.pause_button)
        self.solution_log = QTextEdit()
        layout.addWidget(self.solution_log)
        self.distance_matrix_display = QTextEdit()
        self.distance_matrix_display.setReadOnly(True)
        layout.addWidget(self.distance_matrix_display)
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_visualization)
        self.is_paused = False
        self.show()
        self.display_distance_matrix()

    def start_optimization(self):
        self.start_button.setEnabled(False)
        self.timer.start(30)
        self.update_visualization()

    def pause_optimization(self):
        if self.is_paused:
            self.timer.start()
            self.pause_button.setText("Pause")
        else:
            self.timer.stop()
            self.pause_button.setText("Resume")
        self.is_paused = not self.is_paused

    def update_visualization(self):
        if self.current_iteration < self.max_iterations:
            self.tsp_solver.hill_climbing()
            self.current_iteration += 1
            self.ax[0].clear()
            self.ax[1].clear()
            self.draw_cities()
            self.draw_tour()
            self.draw_optimization_history()
            tour_length_text = f"Tour Length: {self.tsp_solver.tour_length:.2f}"
            temperature_text = f"Temperature: {self.tsp_solver.temperature:.2f}"
            tour_text = f"Tour: {self.display_tour_info(self.tsp_solver.tour)}"
            self.ax[0].text(0.05, 1.1, tour_length_text, fontsize=10, transform=self.ax[0].transAxes)
            self.ax[0].text(0.05, 1.05, temperature_text, fontsize=10, transform=self.ax[0].transAxes)
            self.ax[0].text(0.5, 1.1, tour_text, fontsize=10, transform=self.ax[0].transAxes)
            self.update_solution_log()
        else:
            self.timer.stop()
            self.ax[0].text(0.5, 1.05, f"Best Tour Length: {self.tsp_solver.best_tour_length:.2f}", fontsize=10, transform=self.ax[0].transAxes)
            self.draw_best_tour(self.tsp_solver.best_tour)
        self.canvas.draw()

    def update_solution_log(self):
        log_text = "Solution Log:\n"
        for tour, length in self.tsp_solver.optimization_history:
            tour_text = " ➔ ".join(self.tsp_solver.cities[i][2] for i in tour)
            log_text += f"Tour: {tour_text} | Length: {length:.2f}\n"
        self.solution_log.setText(log_text)

    def display_tour_info(self, tour):
        tour_cities = [self.tsp_solver.cities[i][2] for i in tour]
        return " ➔ ".join(tour_cities)

    def draw_cities(self):
        self.ax[0].grid(True)
        self.ax[0].set_xlim(0, 1)
        self.ax[0].set_ylim(0, 1)
        self.ax[0].set_xlabel('X')
        self.ax[0].set_ylabel('Y')
        for city in self.tsp_solver.cities:
            self.ax[0].plot(city[0], city[1], 'ko', markersize=10)
            self.ax[0].text(city[0] + 0.01, city[1] + 0.01, city[2], fontsize=8)

    def draw_tour(self):
        for i in range(len(self.tsp_solver.tour) - 1):
            self.ax[0].plot([self.tsp_solver.cities[self.tsp_solver.tour[i]][0], self.tsp_solver.cities[self.tsp_solver.tour[i+1]][0]],
                            [self.tsp_solver.cities[self.tsp_solver.tour[i]][1], self.tsp_solver.cities[self.tsp_solver.tour[i+1]][1]], 'r-', linewidth=2)
        self.ax[0].plot([self.tsp_solver.cities[self.tsp_solver.tour[-1]][0], self.tsp_solver.cities[self.tsp_solver.tour[0]][0]],
                        [self.tsp_solver.cities[self.tsp_solver.tour[-1]][1], self.tsp_solver.cities[self.tsp_solver.tour[0]][1]], 'r-', linewidth=2)

    def draw_optimization_history(self):
        if self.tsp_solver.optimization_history:
            iterations = range(1, len(self.tsp_solver.optimization_history) + 1)
            self.ax[1].plot(iterations, [length for _, length in self.tsp_solver.optimization_history], 'b-')
            self.ax[1].set_xlabel('Iterations')
            self.ax[1].set_ylabel('Tour Length')
            self.ax[1].set_title('Optimization History', fontsize=10)

    def draw_best_tour(self, best_tour):
        if best_tour:
            for i in range(len(best_tour) - 1):
                self.ax[0].plot([self.tsp_solver.cities[best_tour[i]][0], self.tsp_solver.cities[best_tour[i+1]][0]],
                                [self.tsp_solver.cities[best_tour[i]][1], self.tsp_solver.cities[best_tour[i+1]][1]], 'g-', linewidth=2)
            self.ax[0].plot([self.tsp_solver.cities[best_tour[-1]][0], self.tsp_solver.cities[best_tour[0]][0]],
                            [self.tsp_solver.cities[best_tour[-1]][1], self.tsp_solver.cities[self.tsp_solver.best_tour[0]][1]], 'g-', linewidth=2)

    def display_distance_matrix(self):
        self.distance_matrix_display.setText(self.tsp_solver.get_distance_matrix_string())

if __name__ == '__main__':
    app = QApplication(sys.argv)
    tsp_solver = TSPSolver(10)
    tsp_solver.initialize_tour()  # Initialize the tour with New York as start and end
    visualizer = TSPVisualizer(tsp_solver)
    sys.exit(app.exec_())
